In [1]:

!pip install symspellpy

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk import corpus,tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer,WordNetLemmatizer

nltk.download('punkt') # for understanding abbrevations etc
nltk.download('vader_lexicon')   # for sentiment analysis
nltk.download('wordnet')  # clearly defined synonyms,hypernyms,hyponyms
nltk.download('omw-1.4') # open multilingual wordnet for multiple languages not only english

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spellchecker import SpellChecker  # efficient for small datasets
from nltk.tokenize import sent_tokenize,word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

from tqdm import tqdm  # For progress bar

from symspellpy.symspellpy import SymSpell,Verbosity  # spell correcction for large datasets
import pkg_resources

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\achyu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
sia = SentimentIntensityAnalyzer()

In [4]:
sent1 = "i have ever seen this kind of movie"
out = sia.polarity_scores(sent1)
out

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [5]:
sent2 = "This is the worst brand i have ever purchased"
out = sia.polarity_scores(sent2)
out

{'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}

In [6]:
sent3 = "movie is good and movie is bad"
out = sia.polarity_scores(sent3)
out

{'neg': 0.307, 'neu': 0.439, 'pos': 0.254, 'compound': -0.1531}

In [89]:
df = pd.read_csv("D:\\Learnbay\\ML Real Time Project\\Sentiment Analysis\\25-09-24\\tweets.csv")
df

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   label   7920 non-null   int64 
 2   tweet   7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 185.8+ KB


In [91]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [92]:
df1 = df.copy()
df1.head(3)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...


In [93]:
duplicates = df1[df1.duplicated()]
duplicates

,id,label,tweet


In [94]:
df1['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [95]:
df['scores'] = df['tweet'].apply(lambda tweet:sia.polarity_scores(tweet))
df.head(10)

,id,label,tweet,scores
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp..."
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp..."
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp..."
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co..."
5,6,1,iPhone software update fucked up my phone big ...,"{'neg': 0.464, 'neu': 0.536, 'pos': 0.0, 'comp..."
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou..."
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
8,9,0,Bout to go shopping again listening to music #...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [96]:
compound_scores = []

for index,row in df.iterrows():
    compound_score = row['scores']['compound']
    compound_scores.append(round(compound_score,2))

df['compound_scores'] = compound_scores
df.head(10)

,id,label,tweet,scores,compound_scores
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51
5,6,1,iPhone software update fucked up my phone big ...,"{'neg': 0.464, 'neu': 0.536, 'pos': 0.0, 'comp...",-0.83
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",0.57
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00
8,9,0,Bout to go shopping again listening to music #...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00


In [97]:
df['new_compound_score'] = df['compound_scores'].apply(lambda score:0 if score>0 else 1)
df.head(10)

,id,label,tweet,scores,compound_scores,new_compound_score
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0
5,6,1,iPhone software update fucked up my phone big ...,"{'neg': 0.464, 'neu': 0.536, 'pos': 0.0, 'comp...",-0.83,1
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",0.57,0
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1
8,9,0,Bout to go shopping again listening to music #...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1


In [98]:
print(confusion_matrix(df['label'],df['new_compound_score']))
print()
print(classification_report(df['label'],df['new_compound_score']))
print()
print(accuracy_score(df['label'],df['new_compound_score']))

[[2587 3307]
 [ 583 1443]]

              precision    recall  f1-score   support

           0       0.82      0.44      0.57      5894
           1       0.30      0.71      0.43      2026

    accuracy                           0.51      7920
   macro avg       0.56      0.58      0.50      7920
weighted avg       0.69      0.51      0.53      7920


0.5088383838383839


# ML Approach

## 1.Text Cleanup

In [99]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+',"",text,flags = re.MULTILINE)
    text = re.sub(r'\@w+|\#',"",text)
    text = re.sub(r'\d+',"",text)
    text = re.sub(r'[^\w\s]',"",text)
    text = text.lower().strip()
    return text

df['cleaned_tweet'] = df['tweet'].apply(clean_text)
df.head(5)


,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...


## 2. Word Segementation

In [100]:
## The above segmentation is taking too much time

## 3. Spell Correction

In [101]:
# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Correct usage of pkg_resources.resource_filename
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

# Load the dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Function for correcting text
def correct_spelling_symspell(text):
    words = text.split()
    corrected_words = []
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            # Use the top suggestion
            corrected_words.append(suggestions[0].term)
        else:
            # If no suggestion, use the original word
            corrected_words.append(word)
    return ' '.join(corrected_words)

tqdm.pandas()
df['Corrected_tweet'] = df['cleaned_tweet'].progress_apply(correct_spelling_symspell)
df.head(5)

100%|████████████████████████████████████████████████████████████████████████████| 7920/7920 [00:05<00:00, 1553.12it/s]


,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet,Corrected_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...,finally a transparent silicon case thanks to m...
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...,in wired i know in george i was made that way ...
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...,what amazing service apple wont even talk to m...


## 4. Tokenization

In [102]:
from nltk.tokenize import word_tokenize
df['tokenize'] = df['Corrected_tweet'].apply(word_tokenize)
df.head(5)

,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet,Corrected_tweet,tokenize
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...,"[fingerprint, pregnancy, test, android, apps, ..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...,finally a transparent silicon case thanks to m...,"[finally, a, transparent, silicon, case, thank..."
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...,we love this would you go talk makememories un...,"[we, love, this, would, you, go, talk, makemem..."
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...,in wired i know in george i was made that way ...,"[in, wired, i, know, in, george, i, was, made,..."
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...,what amazing service apple wont even talk to m...,"[what, amazing, service, apple, wont, even, ta..."


## 5. Stopwords

In [103]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = set(stop_words)
print(stop_words)

{'how', 'he', 'be', 'who', 'my', 'the', 'while', 'that', 'with', 'because', 'over', 'they', 'against', 'didn', 'those', 'this', 'as', 'our', "you're", 'being', 'we', 'is', 'haven', 'been', 'your', 'and', "didn't", "shan't", 'she', 'doing', 'after', 'here', 're', 'from', 'on', 'then', 'which', 's', 'most', 'hers', 'do', 'below', 'under', 'other', 'up', 'off', 'now', 'doesn', 'not', 'his', 'hadn', 'i', 'these', "don't", "you'd", 'before', 'should', 'll', "shouldn't", 'it', 'if', 'above', 'into', 'too', 'her', 'were', 'what', 'both', 'can', 'such', "should've", 'd', 'me', 'ma', 'ain', 'own', 'ourselves', 'himself', 'aren', 'so', 't', 'or', "hadn't", "haven't", "needn't", 'just', 'same', 'its', 'myself', 'whom', 'themselves', 'but', 'through', 'few', 'no', 'wasn', 'are', 'further', 'does', 'some', 'will', "it's", 'than', 'shouldn', 'down', 'by', 'again', "aren't", 'isn', 'did', 'herself', 'ours', 'between', 'itself', 'a', "wasn't", 'm', 'o', 'each', 'out', 'y', 'weren', 'shan', 'won', "cou

In [104]:
'not' in stop_words

True

In [105]:
any(word in stop_words for word in ['not','no'])

True

In [106]:
'no' in stop_words

True

In [107]:
# remove 'not' from stopwords
for word in ['not','no']:
    stop_words.discard(word)


In [108]:
any(word in stop_words for word in ['not','no'])

False

In [109]:
df['cleaned_stopwords'] = df['tokenize'].apply(lambda tokens:[word for word in tokens if word not in stop_words])
df.head(5)

,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet,Corrected_tweet,tokenize,cleaned_stopwords
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...,"[fingerprint, pregnancy, test, android, apps, ...","[fingerprint, pregnancy, test, android, apps, ..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...,finally a transparent silicon case thanks to m...,"[finally, a, transparent, silicon, case, thank...","[finally, transparent, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...,we love this would you go talk makememories un...,"[we, love, this, would, you, go, talk, makemem...","[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...,in wired i know in george i was made that way ...,"[in, wired, i, know, in, george, i, was, made,...","[wired, know, george, made, way, iphone, cute,..."
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...,what amazing service apple wont even talk to m...,"[what, amazing, service, apple, wont, even, ta...","[amazing, service, apple, wont, even, talk, qu..."


## 6. Lemmatization

In [110]:
lemmatizer = WordNetLemmatizer()

df['cleaned_lemmatized'] = df['cleaned_stopwords'].apply(lambda tokens:[lemmatizer.lemmatize(word) for word in tokens])
df.head(10)

,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet,Corrected_tweet,tokenize,cleaned_stopwords,cleaned_lemmatized
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...,"[fingerprint, pregnancy, test, android, apps, ...","[fingerprint, pregnancy, test, android, apps, ...","[fingerprint, pregnancy, test, android, apps, ..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...,finally a transparent silicon case thanks to m...,"[finally, a, transparent, silicon, case, thank...","[finally, transparent, silicon, case, thanks, ...","[finally, transparent, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...,we love this would you go talk makememories un...,"[we, love, this, would, you, go, talk, makemem...","[love, would, go, talk, makememories, unplug, ...","[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...,in wired i know in george i was made that way ...,"[in, wired, i, know, in, george, i, was, made,...","[wired, know, george, made, way, iphone, cute,...","[wired, know, george, made, way, iphone, cute,..."
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...,what amazing service apple wont even talk to m...,"[what, amazing, service, apple, wont, even, ta...","[amazing, service, apple, wont, even, talk, qu...","[amazing, service, apple, wont, even, talk, qu..."
5,6,1,iPhone software update fucked up my phone big ...,"{'neg': 0.464, 'neu': 0.536, 'pos': 0.0, 'comp...",-0.83,1,iphone software update fucked up my phone big ...,iphone software update fucked up my phone big ...,"[iphone, software, update, fucked, up, my, pho...","[iphone, software, update, fucked, phone, big,...","[iphone, software, update, fucked, phone, big,..."
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",0.57,0,happy for us instapic instadaily us sony xper...,happy for us instapic instadaily us sony opera...,"[happy, for, us, instapic, instadaily, us, son...","[happy, us, instapic, instadaily, us, sony, op...","[happy, u, instapic, instadaily, u, sony, oper..."
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,new type c charger cable uk bay amazon etsy ...,new type a charger cable us bay amazon easy ne...,"[new, type, a, charger, cable, us, bay, amazon...","[new, type, charger, cable, us, bay, amazon, e...","[new, type, charger, cable, u, bay, amazon, ea..."
8,9,0,Bout to go shopping again listening to music #...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,bout to go shopping again listening to music i...,bout to go shopping again listening to music i...,"[bout, to, go, shopping, again, listening, to,...","[bout, go, shopping, listening, music, iphone,...","[bout, go, shopping, listening, music, iphone,..."
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,photo fun selfie pool water sony camera picoft...,photo fun self pool water sony camera picofthe...,"[photo, fun, self, pool, water, sony, camera, ...","[photo, fun, self, pool, water, sony, camera, ...","[photo, fun, self, pool, water, sony, camera, ..."


In [111]:
# There are reviews other than english

def keep_english(text):
    if isinstance(text,str):
        return re.sub(r'[^a-zA-Z\s]','',text)
    return ''    

df['Complete_clean_data'] =df['cleaned_lemmatized'].apply(keep_english)

In [112]:
df.head(5)

,id,label,tweet,scores,compound_scores,new_compound_score,cleaned_tweet,Corrected_tweet,tokenize,cleaned_stopwords,cleaned_lemmatized,Complete_clean_data
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.00,1,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...,"[fingerprint, pregnancy, test, android, apps, ...","[fingerprint, pregnancy, test, android, apps, ...","[fingerprint, pregnancy, test, android, apps, ...",
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",0.71,0,finally a transparant silicon case thanks to ...,finally a transparent silicon case thanks to m...,"[finally, a, transparent, silicon, case, thank...","[finally, transparent, silicon, case, thanks, ...","[finally, transparent, silicon, case, thanks, ...",
2,3,0,We love this! Would you go? #talk #makememorie...,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.67,0,we love this would you go talk makememories un...,we love this would you go talk makememories un...,"[we, love, this, would, you, go, talk, makemem...","[love, would, go, talk, makememories, unplug, ...","[love, would, go, talk, makememories, unplug, ...",
3,4,0,I'm wired I know I'm George I was made that wa...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.23,0,im wired i know im george i was made that way ...,in wired i know in george i was made that way ...,"[in, wired, i, know, in, george, i, was, made,...","[wired, know, george, made, way, iphone, cute,...","[wired, know, george, made, way, iphone, cute,...",
4,5,1,What amazing service! Apple won't even talk to...,"{'neg': 0.172, 'neu': 0.574, 'pos': 0.254, 'co...",0.51,0,what amazing service apple wont even talk to m...,what amazing service apple wont even talk to m...,"[what, amazing, service, apple, wont, even, ta...","[amazing, service, apple, wont, even, talk, qu...","[amazing, service, apple, wont, even, talk, qu...",


# Encoding

In [113]:
# OHE

from sklearn.feature_extraction.text import TfidfVectorizer
df['cleaned_string'] = df['cleaned_lemmatized'].apply(lambda x:" ".join(x))

tfidf_vectorizer = TfidfVectorizer(max_features =10000,min_df = 2,max_df =0.95)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_string'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),columns = tfidf_vectorizer.get_feature_names_out())

print(tfidf_df.head(5))

   ___  _silly_u   ab  aba  abercrombie  abfadofficial  ability  able  \
0  0.0       0.0  0.0  0.0          0.0            0.0      0.0   0.0   
1  0.0       0.0  0.0  0.0          0.0            0.0      0.0   0.0   
2  0.0       0.0  0.0  0.0          0.0            0.0      0.0   0.0   
3  0.0       0.0  0.0  0.0          0.0            0.0      0.0   0.0   
4  0.0       0.0  0.0  0.0          0.0            0.0      0.0   0.0   

   aboutalook  absolute  ...  zone  zoo  zoom  αγάπη  андроид  أطفال  سعادة  \
0         0.0       0.0  ...   0.0  0.0   0.0    0.0      0.0    0.0    0.0   
1         0.0       0.0  ...   0.0  0.0   0.0    0.0      0.0    0.0    0.0   
2         0.0       0.0  ...   0.0  0.0   0.0    0.0      0.0    0.0    0.0   
3         0.0       0.0  ...   0.0  0.0   0.0    0.0      0.0    0.0    0.0   
4         0.0       0.0  ...   0.0  0.0   0.0    0.0      0.0    0.0    0.0   

   فرح  センター試験  手机摄影  
0  0.0     0.0   0.0  
1  0.0     0.0   0.0  
2  0.0     0.0   

In [114]:
df['cleaned_lemmatized']

0       [fingerprint, pregnancy, test, android, apps, ...
1       [finally, transparent, silicon, case, thanks, ...
2       [love, would, go, talk, makememories, unplug, ...
3       [wired, know, george, made, way, iphone, cute,...
4       [amazing, service, apple, wont, even, talk, qu...
                              ...                        
7915    [live, loud, low, liveoutloud, self, smile, so...
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, year, old, neighbour, ipad, ...
7918    [finally, got, smart, pocket, wifi, stay, conn...
7919    [apple, barcelona, apple, store, bin, barcelon...
Name: cleaned_lemmatized, Length: 7920, dtype: object